# General Settings:

Directory Data:

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime


import ribo_util
import ribo_main
import ribo_analysis
import ribo_plot
import ribo_coords

import os

# File names can be manually added to list, or read as .csv with header == Library
# Library names must be the same as the fastq file name

path_pc     = '/home/gyrase/datatank/Bikmetov/toxin_profiling/2018_Bacterial_Pipeline_riboseq-clean_first/results/'  # location of folder for ribosome profiling
path_script = '/home/gyrase/datatank/Bikmetov/toxin_profiling/2018_Bacterial_Pipeline_riboseq-clean_first/'

all_files  = [x for x in os.listdir(path_pc + '/libraries/FASTQ/')]
file_csv   = path_script  + 'Library_names.csv'  # alternatively, provide .csv with filenames
'''
all_files  = []                              # sample name, same as FASTQ filename
file_csv   = path_pc  + 'Library_names.csv'  # alternatively, provide .csv with filenames

library_id = pd.read_csv(file_csv)

for fname in library_id.Library:
    all_files.append(fname)'''
print all_files

inputs = {}
inputs['files'] = []
inputs['multiprocess'] = 'yes'
inputs['threads']      = 20   # CPU information for multithreading applications
inputs['cores']        = 10

paths_in = {}
paths_in['path_gff']        = path_pc + 'annotations/Coli/Coli.gff'
paths_in['path_gff_dict']   = path_pc + 'annotations/Coli/Coli_dict'       # this will be made from the GFF file
paths_in['path_badgenes']   = path_pc + 'annotations/Coli/bad_genes.csv'   # list of genes to exclude
paths_in['path_annotation'] = path_pc + 'annotations/Coli/annotation.csv'  # adds gene description to GFF_dict 

# Outputs from analysis will be placed in the foillowing directories:
paths_out = {}
paths_out['path_density']      = path_pc  + 'libraries/density/density/'  # Created from Ribo_Density
paths_out['path_log']          = path_pc  + 'libraries/density/logs/'
paths_out['path_analysis_log'] = path_pc  + 'libraries/analysis/logs/'
paths_out['path_analysis']     = path_pc  + 'libraries/analysis/individual/'
paths_out['path_figures']      = path_pc  + 'libraries/figures/'

    
# Check inputs, create output paths
step = 'analysis'                                # density or analysis
ribo_util.check_inputs(inputs, paths_in, step)   # will remove file from analysis for not having a density file
ribo_util.createpath(inputs, paths_out, all_files)          # create output paths


['T62.filter.fastq', 'TacT3.filter.fastq', 'Ita3.filter.fastq', 'Control.filter.fastq']


# Analysis Settings

Set settings values for the various analyses

In [20]:
# General Settings 
settings = {}
settings['minlength'] = 15    # minimum read lenght
settings['maxlength'] = 40    # maximum read length
settings['shift']     = 11    # A-site shift
settings['gff_extra'] = 50    # Number of nt added before start seq and after stop seq in GFF_dict
settings['threshold'] = 1    # in reads per codon
settings['alignment'] = '3'   # '3' or '5'

# Avggenes Settings
settings['length_out_ORF'] = 50
settings['length_in_ORF']  = 100        # genes shorter than this are excluded
settings['density_type']   = 'reads'    # 'reads' or 'rpm' 
settings['equal_weight']   = 'yes'      # 'yes' or 'no', if yes, change density_type to reads -- faster
settings['next_gene']      = 25         # genes closer than this are removed from start and stop 

# Pausescore settings
settings['A_site shift']    = -11
settings['plot_upstream']   = 40
settings['plot_downstream'] = 50
settings['start_trim']      = 50
settings['stop_trim']       = 20
settings['frameshift']      = 0

# Genelist settings
settings['subgroup'] = 'none'

# Pausescore waves settings
settings['plot_upstream_wave']   = 106
settings['plot_downstream_wave'] = 106
settings['next_codon']           = 'yes'

# Motif analysis
settings['motif_length'] = 9


# Create Annotation dictionary from GFF

Pipeline uses a simplified dictionary of the GFF for speed. This function will strip the GFF to the gene name, start and stop position, and sequence of the gene

This needs to be run only once, so you can comment it off once it does run


In [21]:
gff_settings = {}
gff_settings['path_out']         = 0             # set to 0 to output in annotations folder
gff_settings['feat_of_interest'] = 'all'         # all, CDS, tRNA, rRNA: recommend using all
gff_settings['name_qual']        = 'Name'        # GFF gene qualifier
gff_settings['name_qual_alt']    = 'ID'          # Secondary GFF gene qualifier if no name is present
gff_settings['remove_genes']     = 'yes'         # remove hard to align genes listed in bad_genes.csv
gff_settings['gff_extra']         = 50           # additional sequence upstream and downstream of gene (set to 50)

#GFF_conversion = ribo_util.GFF_to_dict(paths_in, gff_settings)


In [22]:
lib_count = len(all_files)
lib_index = 0 
lib_runs  = 8  # limits number of samples processed, scale with RAM imitations

for loops in range(0, lib_count / lib_runs + 2):

    if lib_index < lib_count:
        print "Started  at " + str(datetime.now())
        if lib_count - lib_index >= lib_runs:
            inputs['files'] = [all_files[i] for i in range(lib_index, lib_index + lib_runs)]

            print inputs['files']
            
            if not 'gff_dict' in globals(): 
                gff_dict, plus_dict, minus_dict = ribo_util.loadlargePickles(inputs, settings, paths_in, paths_out)
            else: 
                gff_dict, plus_dict, minus_dict = ribo_util.loadlargePickles(inputs, settings, paths_in, paths_out)
                        
            asymmetry_analysis = ribo_coords.asymmetry(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            
        else:
            inputs['files'] = [all_files[i] for i in range(lib_index, lib_count)]
            
            print inputs['files']    
            if not 'gff_dict' in globals(): 
                gff_dict, plus_dict, minus_dict = ribo_util.loadlargePickles(inputs, settings, paths_in, paths_out)
            else: 
                gff_dict, plus_dict, minus_dict = ribo_util.loadlargePickles(inputs, settings, paths_in, paths_out)
                
            asymmetry_analysis = ribo_coords.asymmetry(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            
    else:
        continue
    lib_index += lib_runs  
print "Finished  at " + str(datetime.now())

Started  at 2020-01-24 16:15:16.842248
['T62.filter.fastq', 'TacT3.filter.fastq', 'Ita3.filter.fastq', 'Control.filter.fastq']
Started asymmetry analysis at 2020-01-24 16:17:32.802675
42360136.0
[7115274. 7029851. 5286136. 4618415. 3651400. 3253557. 3293794. 2854008.
 2803412. 2454289.]
13150128.0
[3405642. 2448826. 1679751. 1300325.  888547.  795895.  795856.  640197.
  600110.  594979.]
5071904.0
[913542. 759410. 666052. 535856. 451255. 395622. 392254. 361796. 338128.
 257989.]
16846500.0
[2278037. 1905547. 1778740. 1653611. 1502624. 1618224. 1560043. 1551383.
 1499831. 1498460.]
Finished asymmetry analysis at 2020-01-24 16:17:42.130581
Finished  at 2020-01-24 16:17:42.131013


In [1]:
import pickle
from scipy.stats.mstats import gmean
from skbio.stats import composition

In [2]:
samples = ['Control', 'TacT2']

In [3]:
data = {}
for x in samples:
    with open('../%s_solo' % x, 'r') as inf:
        data[x] = pd.DataFrame.from_dict(pickle.load(inf)).T
        index = [np.prod(y) != 0 for y in data[x][0]]
        data[x] = data[x][index]

IOError: [Errno 2] No such file or directory: '../Control_solo'

In [4]:
both = set(data['Control'].index) & set(data['TacT2'].index)
len(both)

KeyError: 'Control'

In [ ]:
gmeans = {}
for x in samples:
    df = data[x].filter(items = both, axis = 0).sort_index()[0]
    per_cent = composition.closure(df)
    ilr = df.apply(composition.ilr)
    df = np.stack(df.to_numpy())
    ilr = np.stack(ilr.to_numpy())
    np.savetxt("%s_paired_raw.csv" % x, df, delimiter=",")
    np.savetxt("%s_paired_ilr.csv" % x, ilr, delimiter=",")
    gmeans[x] = gmean(per_cent, axis = 0)

In [5]:
gmeans

NameError: name 'gmeans' is not defined